In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/MyDrive/Colab Notebooks/animal_data.zip"

'/content/drive/MyDrive/Colab Notebooks/animal_data.zip'


In [ ]:
!unzip -o "/content/drive/MyDrive/Colab Notebooks/animal_data.zip" -d data

Archive:  /content/drive/MyDrive/Colab Notebooks/animal_data.zip
  inflating: data/__MACOSX/._animal_data  
  inflating: data/__MACOSX/animal_data/._Cat  
  inflating: data/__MACOSX/animal_data/._Dog  
  inflating: data/__MACOSX/animal_data/._Dolphin  
  inflating: data/__MACOSX/animal_data/._Giraffe  
  inflating: data/__MACOSX/animal_data/._Bear  
  inflating: data/__MACOSX/animal_data/._Zebra  
  inflating: data/__MACOSX/animal_data/._Panda  
  inflating: data/__MACOSX/animal_data/._Tiger  
  inflating: data/__MACOSX/animal_data/._Bird  
  inflating: data/__MACOSX/animal_data/._Kangaroo  
  inflating: data/__MACOSX/animal_data/._Horse  
  inflating: data/__MACOSX/animal_data/._Cow  
  inflating: data/__MACOSX/animal_data/._Deer  
  inflating: data/__MACOSX/animal_data/._Lion  
  inflating: data/__MACOSX/animal_data/._Elephant  
  inflating: data/animal_data/Cat/Cat_15.jpg  
  inflating: data/__MACOSX/animal_data/Cat/._Cat_15.jpg  
  inflating: data/animal_data/Cat/Cat_29_1.jpg  
  i

In [ ]:
!ls data
!ls data/animal_data

animal_data  __MACOSX
Bear  Cat  Deer  Dolphin   Giraffe  Kangaroo  Panda  Zebra
Bird  Cow  Dog	 Elephant  Horse    Lion      Tiger


In [ ]:
transform = transforms.Compose([transforms.Resize((128,128)),transforms.ToTensor()])
full_data = datasets.ImageFolder(root = "data/animal_data",transform = transform)
n = len(full_data)
n_train = int(0.8*n)
n_test = n - n_train
train_data, test_data = random_split(full_data, [n_train, n_test])

In [ ]:
train_loader = DataLoader(train_data, batch_size = 32, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 32, shuffle = False)

In [ ]:
from torch._C import device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class CNN(nn.Module):
  def __init__(self, num_classes):
    super().__init__()
    self.conv1 = nn.Conv2d(3,16,kernel_size = 3, padding = 1)
    self.conv2  = nn.Conv2d(16,32, kernel_size = 3, padding = 1)
    self.pool = nn.MaxPool2d(2,2)
    self.fc1 = nn.Linear(32*32*32, 128)
    self.fc2 = nn.Linear(128, num_classes)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(x.size(0), -1)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x
# Number of classes from ImageFolder
num_classes = len(train_data.dataset.classes)

#Create model and move to device
model = CNN(num_classes = num_classes).to(device)
print(model)

Using device: cpu
CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=32768, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=15, bias=True)
)


In [ ]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

num_epochs = 10

for epoch in range(num_epochs):
  model.train()
  running_loss = 0

  for images, labels in train_loader:
    images = images.to(device)
    labels = labels.to(device)
    # 1) Forward pass
    outputs = model(images)
    # 2) compute loss
    loss = criterion(outputs, labels)
    # Backward + o[timize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  avg_loss = running_loss / len(train_loader)
  len(train_loader)
  print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

Epoch 1/10, Loss: 2.7096
Epoch 2/10, Loss: 2.7090
Epoch 3/10, Loss: 2.7091
Epoch 4/10, Loss: 2.7083
Epoch 5/10, Loss: 2.7087
Epoch 6/10, Loss: 2.7088
Epoch 7/10, Loss: 2.7080
Epoch 8/10, Loss: 2.7081
Epoch 9/10, Loss: 2.7085
Epoch 10/10, Loss: 2.7089


In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)

    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

    test_accuracy  = 100*correct/total
    print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 6.25%
Test Accuracy: 6.25%
Test Accuracy: 4.17%
Test Accuracy: 4.69%
Test Accuracy: 6.25%
Test Accuracy: 6.25%
Test Accuracy: 5.80%
Test Accuracy: 5.47%
Test Accuracy: 5.21%
Test Accuracy: 5.31%
Test Accuracy: 5.97%
Test Accuracy: 5.73%
Test Accuracy: 5.66%
